In [1]:
import xarray as xr
# filepath_zarr = '../data/dwd/processed/radolan_berlin_2022-01-01_2025-04-30.zarr'
filepath_zarr = '../data/dwd/processed/radolan_berlin_2022-01-01_2022-02-28.zarr'

data_series_loaded = xr.open_zarr(filepath_zarr)
data_series_loaded

<xarray.Dataset> Size: 238kB
Dimensions:        (time: 58, x: 30, y: 33)
Coordinates:
    lat            (x, y) float32 4kB dask.array<chunksize=(30, 33), meta=np.ndarray>
    lon            (x, y) float32 4kB dask.array<chunksize=(30, 33), meta=np.ndarray>
  * time           (time) datetime64[ns] 464B 2022-01-01T12:50:00 ... 2022-02...
Dimensions without coordinates: x, y
Data variables:
    precipitation  (time, x, y) float32 230kB dask.array<chunksize=(58, 30, 33), meta=np.ndarray>

In [2]:
# Examine the dataset structure
print("Dataset dimensions:", data_series_loaded.sizes)
print("\nCoordinates:", list(data_series_loaded.coords))
print("\nData variables:", list(data_series_loaded.data_vars))
print("\nTime range:", data_series_loaded.time.values[[0, -1]])
print("\nLatitude range:", data_series_loaded.lat.values.min(), "to",
      data_series_loaded.lat.values.max())
print("\nLongitude range:", data_series_loaded.lon.values.min(), "to",
      data_series_loaded.lon.values.max())

Dataset dimensions: Frozen({'time': 58, 'x': 30, 'y': 33})

Coordinates: ['lat', 'lon', 'time']

Data variables: ['precipitation']

Time range: ['2022-01-01T12:50:00.000000000' '2022-02-27T12:50:00.000000000']

Latitude range: 52.39097 to 52.65755

Longitude range: 13.1390085 to 13.617209


In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely.geometry import Point
import warnings

warnings.filterwarnings('ignore')

In [4]:
# Prepare data for visualization
def prepare_precipitation_data(data_series):
    """
    Convert xarray dataset to pandas DataFrame for plotting
    """
    # Get coordinates
    lat_coords = data_series.lat.values
    lon_coords = data_series.lon.values
    precipitation_data = data_series.precipitation.values

    # Create a list to store all data points
    data_list = []

    # Loop through each time step
    for t_idx, time_val in enumerate(data_series.time.values):
        # Loop through each grid point
        for x_idx in range(precipitation_data.shape[1]):  # x dimension
            for y_idx in range(precipitation_data.shape[2]):  # y dimension
                lat = lat_coords[x_idx, y_idx]
                lon = lon_coords[x_idx, y_idx]
                precip = precipitation_data[t_idx, x_idx, y_idx]

                data_list.append({
                    'time': pd.to_datetime(time_val),
                    'latitude': lat,
                    'longitude': lon,
                    'precipitation': precip,
                    'x_idx': x_idx,
                    'y_idx': y_idx
                })

    return pd.DataFrame(data_list)


print("Preparing precipitation data...")
# Take a subset of data for faster processing (every 7 days to start)
subset_data = data_series_loaded.isel(time=slice(0, None, 7))  # Every 7th day
df_precip = prepare_precipitation_data(subset_data)

print(f"Data prepared! Shape: {df_precip.shape}")
print(f"Date range: {df_precip['time'].min()} to {df_precip['time'].max()}")
print(
    f"Precipitation range: {df_precip['precipitation'].min():.2f} to {df_precip['precipitation'].max():.2f}"
)
df_precip.head()

Preparing precipitation data...
Data prepared! Shape: (8910, 6)
Date range: 2022-01-01 12:50:00 to 2022-02-26 12:50:00
Precipitation range: 0.00 to 12.20


,time,latitude,longitude,precipitation,x_idx,y_idx
0,2022-01-01 12:50:00,52.407196,13.139009,3.5,0,0
1,2022-01-01 12:50:00,52.406723,13.153150,3.9,0,1
2,2022-01-01 12:50:00,52.406246,13.167290,4.1,0,2
3,2022-01-01 12:50:00,52.405769,13.181430,3.7,0,3
4,2022-01-01 12:50:00,52.405289,13.195569,3.8,0,4


In [ ]:
# Create interactive map with time slider
def create_precipitation_map(df):
    """
    Create an interactive map with precipitation data and time slider
    """
    # Add date string for grouping
    df['date_str'] = df['time'].dt.strftime('%Y-%m-%d')

    # Create the figure
    fig = px.scatter_mapbox(
        df,
        lat="latitude",
        lon="longitude",
        color="precipitation",
        size="precipitation",
        animation_frame="date_str",
        hover_data={
            'latitude': ':.4f',
            'longitude': ':.4f',
            'precipitation': ':.2f',
            'date_str': False
        },
        color_continuous_scale="Blues",
        size_max=15,
        zoom=10,
        mapbox_style="open-street-map",
        title="Berlin Precipitation Data - Interactive Map with Time Slider",
        labels={
            'precipitation': 'Precipitation (mm)',
            'latitude': 'Latitude',
            'longitude': 'Longitude'
        })

    # Update layout
    # fig.update_layout(height=700,
    #                   margin={
    #                       "r": 0,
    #                       "t": 50,
    #                       "l": 0,
    #                       "b": 0
    #                   },
    #                   coloraxis_colorbar=dict(title="Precipitation (mm)"))

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            center=dict(lat=52.5200, lon=13.4050),  # Berlin coordinates
            zoom=10),
        height=700,
        margin={
            "r": 10,
            "t": 60,
            "l": 10,
            "b": 10
        },
        coloraxis_colorbar=dict(title="Precipitation (mm)"))

    # Update animation settings
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"][
        "duration"] = 300

    return fig


# Create the map
print("Creating interactive map...")
fig_map = create_precipitation_map(df_precip)

# Show the map
fig_map.show()

import os

# Save the figure for later use in Streamlit
output_dir = '../src/fe/assets/'
os.makedirs(output_dir, exist_ok=True)
fig_map.write_json(os.path.join(output_dir, 'precipitation_plot.json'))

print(f"\nMap saved to:")
print(f"  JSON: {os.path.join(output_dir, 'precipitation_plot.json')}")

Creating interactive map...



Map saved to:
  JSON: ../src/fe/assets/precipitation_plot.json
